In [1]:
import sys; sys.dont_write_bytecode=1
import warnings
from typing import Literal as Lit
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import deps as _
import modules as m
import Common
import gc
from importlib import reload; reload(_)

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [2]:
TICKERS = _.TECH_7
TF      = 'Y'
SRC     = '1mo'
UNTIL   = '35y'
GRAN    = 12
ANUM    = 1
PRDS    = 1, 3, 5, 10, 20, 40 
SLGS    = '1Y', '3Y', '5Y', '10Y', '20Y', '40Y'


LI_RNK_1 = Common.LI_RNK_1[TF]
LI_RNK_2 = Common.LI_RNK_2[TF]
TP_RNK_1 = Common.TP_RNK_1[TF]
TP_RNK_2 = Common.TP_RNK_2[TF]

RNK_1:TP_RNK_1  = 'High Cor 1Y' 
RNK_2:TP_RNK_2  = 'High Gap 1Y'
IND_1, IND_2    = _.np_remove_prefix([RNK_1, RNK_2])
WGHTS           = [1,1]
NORM_WGHTS      = False
RCOMB           = 'High Art'

In [3]:
def GET_SOURCE(toggle, TICKERS, TF, SRC, UNTIL, FULL_CLOCK, start=None):
    if not toggle:  return None
    Src  = m.GET_HIST_DATA(TICKERS, SRC, UNTIL, start)
    Keys = m.MAKE_PAIRS_KEYS(Src)
    Src2 = m.MAKE_PAIRS_SOURCE(Keys, Src, TF, FULL_CLOCK)
    
    del Src, Keys;  gc.collect()
    return Src2

In [4]:
def COMPUTE_DATA(toggle, Src2, GRAN, PRDS, SLGS):
    if not toggle:  return None
    PSts    = m.CALCULATE_PRC_STATS(Src2, GRAN, PRDS, SLGS)
    Vars    = m.CALCULATE_VARIATIONS(Src2, GRAN, PRDS, SLGS)
    VSts    = m.CALCULATE_VAR_STATS(Src2, Vars, GRAN, PRDS, SLGS)
    LS      = m.LONG_SHORT_RETURNS(Src2, Vars, PRDS, SLGS)
    IAbs    = m.CALCULATE_ABSOLUTES(Vars, VSts)
    GVar    = m.CALCULATE_GAP_VARIATION(Src2, IAbs, GRAN, PRDS, SLGS)
    Frames  = m.COMBINE_FRAMES(Src2, LS, PSts, Vars, VSts, IAbs, GVar)
    
    del PSts, Vars, VSts, LS, IAbs, GVar;  gc.collect()

    Rnk_Rtn, Rnk_Cor, Rnk_Inds = m.CALCULATE_RANKS(Frames, PRDS, SLGS)
    Ranks   = m.COMBINE_RANKS(Src2, Rnk_Rtn, Rnk_Cor, Rnk_Inds)
    
    del Src2, Rnk_Rtn, Rnk_Cor, Rnk_Inds;  gc.collect()
    
    Data    = m.COMBINE_DATAS(Frames, Ranks)
    
    del Frames, Ranks;  gc.collect()

    Scrn    = m.SORT_SCREENER(Data)

    del Data;  gc.collect()

    # del PSts, Vars, VSts, LS, IAbs, GVar, Frames, Rnk_Rtn, Rnk_Cor, Rnk_Inds, Ranks, Data;  gc.collect()
    return Scrn

In [5]:
def RANKED_FOLIOS(Scrn, TF, TOP, GRAN, ANUM,  RCOMB, IND_1,IND_2,RNK_1,RNK_2,  WGHTS, NORM_WGHTS):
    Slc         = m.SELECT(Scrn, GRAN)

    del Scrn;  gc.collect()

    RnkComb     = m.RANK_COMBINATION(Slc,  ['N','Ticker.XY'],['Date'],  RNK_1, RNK_2,  WGHTS, NORM_WGHTS)
    InTop       = m.TOP_CLASSIFCATION(RnkComb, RankBy=RCOMB, TOP=TOP)
    Class_RComb = m.COMBINE_CLASS_RCOMB(InTop, RnkComb)

    del InTop, RnkComb;  gc.collect()

    DFrame      = m.INSERT_CLASS_RCOMB_IN_DFRAME(Slc, Class_RComb)

    del Class_RComb;  gc.collect()

    Agg         = m.MAKE_AGGREGATION_KEYS(Slc, TOP)

    del Slc;  gc.collect()

    Agg         = m.ADD_WALLET_PERFORMANCE(Agg, DFrame, TF, ANUM, COLS=[IND_1, IND_2, RNK_1, RNK_2])

    del DFrame;  gc.collect()

    Top         = m.FINAL_RESULTS(Agg, TOP, COLS=[IND_1, IND_2, RNK_1, RNK_2])

    # del Slc, RnkComb, InTop, Class_RComb, DFrame;  gc.collect()
    return Top, Agg